# This notebook, exemplify how to create your F8949 starting from forms 1099 provided from your broker. 

Notice this may need to adapt, depending on the number of pages of the PDF files, and in this particular case we use 2 1099 files, one from Robinhood in PDF and a second 1099 form from Schwabs provided on csv format. You will need to have your own 1099 files. Also, you need to download the 8949 form from the official IRS website. I do not provide such files, to avoid any problem in using not updated versions of the forms.

This code was made using files for 2022 tax year. If you are using it as reference for future years, pay attention to any update on the formats of the different forms, and modify acordinly.


## Disclaimer: This is just an example of how PDF files can be readed, the data of the tables can be manipulated and then used for the purpose the someone needs. You need to modify taking in account the format of your files, and any specific tax rules may apply to your particular case. I am not a tax advisor, and this notebook do not constitute tax advice or tax filing or reporting assistance.

In [ ]:
#pip install tabula-py

In [ ]:
# Uncomment next line to instal PyPDF2 library, after intalled, reset Kernel.
#pip install PyPDF2

In [ ]:
# Uncomment next line to instal reportlab library, after intalled, reset Kernel.
#pip install reportlab 

In [ ]:
# Uncomment next line to instal tabula library, after intalled, reset Kernel.
# pip install tabula-py
# https://stackoverflow.com/questions/48136092/tabula-py-java-version-error
# https://github.com/chezou/tabula-py/issues/54
# and run this lines in terminal
# "sudo mv /usr/bin/java /usr/bin/java-1.6"
# "sudo ln -s '/Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/bin/java' /usr/bin/java"

#### parameters thatthe command tabula use:
tabula.read_pdf?
tabula.read_pdf(
    input_path,
    output_format=None,
    encoding='utf-8',
    java_options=None,
    pandas_options=None,
    multiple_tables=True,
    user_agent=None,
    **kwargs,
)

In [ ]:
#Web Scraping info
#https://oxylabs.io/blog/python-web-scraping
    

In [ ]:
# This function find the symbol of the stock based on the CUSIP which is provided in the PDF files by Robinhood.

import requests
def FindSymboyByCUSIP(CUSIPNumber):

    site = 'https://quotes.fidelity.com/mmnet/SymLookup.phtml?reqforlookup=REQUESTFORLOOKUP&productid=mmnet&isLoggedIn=mmnet&rows=50&for=stock&by=cusip&criteria='+CUSIPNumber+'&submit=Search'
    response = requests.get(site, )

    #print(response.text)
    AllText = response.text
    #word = 'geeks for geeks'
    StringPos1 = AllText.find('SID_VALUE_ID=') + 13
    #StringPos = 16022+13
    StringPos2 = StringPos1+25
    StockSymbol = AllText[StringPos1:StringPos2]
    
    StockSymbol = StockSymbol[0:StockSymbol.find('"')]
    #print(StockSymbol )
    return StockSymbol

#Example of CUSIP and use of the function:
CUSIP = 'Y3894J104'
FindSymboyByCUSIP(CUSIP)

In [ ]:
def ReadPage( df1, df_transactios):

    substring1 = 'CALL'
    substring2 = 'PUT'

    for i in range(len(df1)): 
        line = str(df1['disposed Quantity'][i])
        
        if (line[0:15]!='Security total:'): 
            # Is numeric Check, means is a date transaction line
            string = line[0:1] 
            IsANumber = np.char.isnumeric(string)    
            if (IsANumber == False  ):
                #NewName = str(line)
                Pos1 = line.find('CUSIP:')+6
                Pos2 = Pos1+11
                CUSIP = line[Pos1:Pos2]  
                CUSIP = CUSIP.replace(' ','')
                #print(CUSIP)
                if (CUSIP[0:1] != '/'):
                    CUSIP = CUSIP.replace('/','')
                    # This particular CUSIP is not found, so we set it as a particular case.
                    if (CUSIP == '055474100'): 
                        Sybmol ='BIT'
                    else:
                        Symbol =  FindSymboyByCUSIP(CUSIP)  # Assign the the symbol  
                else:
                    Symbol = line[0:6]
                    Symbol = Symbol[0:Symbol.find(' ')]
                    
                # OPTIONS check 
                if (substring1 in line):
                    OptionC = 'Call' 
                else:
                    if (substring2 in line):
                        OptionC = 'Put' 
                    else: 
                        OptionC = ''
                print (' Symbol ',Symbol, 'Opt: ',OptionC )
                
            else:
                DateSold = line[0:8] 
                if (DateSold[5:8]=='/22'):
                    #print ('Date: ',DateSold)
                    string       = line[9:15]             
                    Quantity     = float(string)
                    grossSold    = float(str(df1['(G)ross or (N)et'][i]).replace(',',''))
                    DateAdquired = df1['acquired'][i]
                    #cost = .replace(',','')
                    Cost         = float( str(df1['other basis'][i]).replace(',','') )
                    Washsale     = df1['disallowed (W)'][i]
                    GainLoss     = float( str(df1['also not reported (Z)'][i]).replace(',',''))
                    comment      = df1['Additional information'][i] 
                    #print ('Date: ',DateSold,  '%3.2f'%Quantity)
                    #print (' %3.2f'%grossSold,' DAdq',DateAdquired,'  %3.2f'%Cost,' W:',Washsale ,' %3.2f'%GainLoss,' C:'+comment)
                    df_temp = pd.DataFrame([[Symbol, OptionC, DateSold, Quantity, grossSold, DateAdquired, Cost, 
                            Washsale , GainLoss, comment]],
                            columns = ['Security','Option','Date_Sold', 'Quantity', 'GrossSale',
                            'DateAdquired', 'Cost', 'Washsale', 'Lost', 'Comment'])
                    df_transactios = df_transactios.append(  df_temp, ignore_index=True)
                    
    return df_transactios

In [ ]:
def FillPage(pdf_path,Out_pdf_path, xcoor, ycoor, DeltaX, DeltaY, df_ST, PageNumber, FormNumber, lendf):

    Col_D = 0.00
    Col_E = 0.00
    Col_G = 0.00
    Col_H = 0.00
    npage = FormNumber

    page_to_merge = PageNumber #Refers to the First page of PDF 
    input_pdf     = PdfFileReader(open(pdf_path, "rb"))
    page_count    = input_pdf.getNumPages()
    #overlay       = {'',''}

    #PAGE 1
    inputpdf_page_to_be_merged = input_pdf.getPage(page_to_merge)
    packet        = io.BytesIO()
    
    c             = Canvas(packet,pagesize=(inputpdf_page_to_be_merged.mediaBox.getWidth(),inputpdf_page_to_be_merged.mediaBox.getHeight()))
    
    print(inputpdf_page_to_be_merged.mediaBox.getWidth(),inputpdf_page_to_be_merged.mediaBox.getHeight())
    print (c)
    if (PageNumber % 2) == 0:
        PageNumberIsEven = True
    else: 
        PageNumberIsEven = False
    
    pending_lines =lendf - (14*npage)
    if (pending_lines)<0:
        pending_lines = 0
    else:
        if (pending_lines>14):
            pending_lines = 14
        
    for i in range(pending_lines): 
        j =i+14*npage
        #for j in range(8):
        if (df_ST['Option'][j]==''):
            Description = str('%3.1f'%(df_ST['Quantity'][j]))+' sh. '+str(df_ST['Security'][j])
        if (df_ST['Option'][j]!=''):
            Description = str('%3.0f'%(df_ST['Quantity'][j]))+str(' '+df_ST['Option'][j])+' Opt. '+str(df_ST['Security'][j])
            if (df_ST['Option'][j]=='Crypto'):
                Description = str('%3.2f'%(df_ST['Quantity'][j]))+' '+str(df_ST['Security'][j])
                if (df_ST['Quantity'][j]<0.001):
                    Description = str('%3.6f'%(df_ST['Quantity'][j]))+' '+str(df_ST['Security'][j])
                
        GrossSale   =  df_ST['GrossSale'].loc[j]
        PriceCost   =  df_ST['Cost'].loc[j]
        GainLost    =  df_ST['Lost'].loc[j]
        
        XX = 10  #+ (i*20)
        YY = 390 - (i*24)  
        #txxyy = 'x='+str(XX)+' y='+str(YY) 
        #print(XX, YY, ' i=',i,  )
        #c.drawString(XX, YY,  txxyy)
         
        
        c.drawString(xcoor + 0*DeltaX,    ycoor - i*DeltaY, Description                    )  #A   
        c.drawString(xcoor + 6.7*DeltaX,  ycoor - i*DeltaY, df_ST['DateAdquired'][j]       )  #B "Date")
        c.drawString(xcoor + 9.3*DeltaX,  ycoor - i*DeltaY, df_ST['Date_Sold'][j]          )  #C "DateSold")
        c.drawString(xcoor + 11.5*DeltaX, ycoor - i*DeltaY, str(f'{GrossSale:10,.2f}')     )  #D "SalePrice")
        c.drawString(xcoor + 15*DeltaX,   ycoor - i*DeltaY, str(f'{PriceCost:10,.2f}')     )  #E "Cost")
        c.drawString(xcoor + 24*DeltaX,   ycoor - i*DeltaY, str(f'{GainLost:10,.2f}')      )  #H "Gain/Loss")
        
        if (PageNumberIsEven):  
            Washsale    =  df_ST['Washsale_Value'].loc[j]
            c.drawString(xcoor+19*DeltaX,ycoor - i*DeltaY, df_ST['Washsale_Code'][j]  )  #F "Code")
            c.drawString(xcoor+20*DeltaX,ycoor - i*DeltaY, str(f'{Washsale:10,.2f}')  )  #G "Ajustment")
            Col_G += df_ST['Washsale_Value'][j]
        
        Col_D += df_ST['GrossSale'][j]
        Col_E += df_ST['Cost'][j] 
        Col_H += df_ST['Lost'][j]

    i = 15
    c.drawString(xcoor+11.5*DeltaX,  ycoor - i*DeltaY, f'{Col_D:10,.2f}')   #D SUM
    c.drawString(xcoor+15*DeltaX,ycoor - i*DeltaY, f'{Col_E:10,.2f}')   #E SUM
    c.drawString(xcoor+20*DeltaX,ycoor - i*DeltaY, f'{Col_G:10,.2f}')   #G SUM
    c.drawString(xcoor+24*DeltaX,ycoor - i*DeltaY, f'{Col_H:10,.2f}')   #H Sum
    c.save()
    packet.seek(0)
    overlay_pdf = PdfFileReader(packet)
    overlay     = overlay_pdf.getPage(0)
    output      = PdfFileWriter()

    for PAGE in range(page_count):

        if PAGE == page_to_merge:
            inputpdf_page_to_be_merged.mergeRotatedTranslatedPage(overlay, 
                    inputpdf_page_to_be_merged.get('/Rotate') or 0, 
                    int(overlay.mediaBox.getWidth()/20), int(overlay.mediaBox.getHeight()/20)) #int(overlay.mediaBox.getWidth()/3)
            output.addPage(inputpdf_page_to_be_merged)
        else:
            Page_in_pdf = input_pdf.getPage(PAGE)
            output.addPage(Page_in_pdf)

    outputStream = open(Out_pdf_path, "wb")
    print (Out_pdf_path)
    output.write(outputStream)
    outputStream.close()
    print ('Totals %5.2f'%Col_D, '%5.2f'%Col_E, '%5.2f'%Col_G, '%5.2f'%Col_H, '\n')

In [ ]:
def WashSaleColumns(dftransactios):
    lendf = len(dftransactios)
    dfWcode  = pd.DataFrame( ['']*lendf, columns = {'Washsale_Code'})
    dfWValue = pd.DataFrame( np.zeros(lendf), columns = {'Washsale_Value'})

    for i in range(lendf):
        li = len(dftransactios['Washsale'][i])
        WSaleCode = str(dftransactios['Washsale'][i])
        value = (str(dftransactios['Washsale'][i])[0:li-2]).replace(',','')  

        if (WSaleCode =='...'):
            WSValue = 0.000
            WSCode  = ' '
        else:
            WSCode ='W'
            WSValue = float(value ) 

        dfWcode.loc[i] = WSCode
        dfWValue.loc[i] = WSValue

    dftransactios['Washsale_Code']  =  dfWcode
    dftransactios['Washsale_Value'] =  dfWValue
    return dftransactios

In [ ]:
# Import the required Module
import tabula as tabula
from pathlib import Path
import pandas as pd
import numpy as np


Path.home()

In [ ]:
# Import the required Lybraries
import tabula as tabula
from pathlib import Path
import pandas as pd
import numpy as np

# Use the path to where the tax form provided by Robinhood is in your computed
# I had the form inside a folder name TaxesForms/2022TaxYear/RobinHood_1099form2022.pdf
Form1099_path = (
    Path.home()
    / "TaxesForms"
    / "2022TaxYear"
    / "RobinHood_1099form2022.pdf"
)

#References:
#from: https://medium.com/analytics-vidhya/how-to-extract-multiple-tables-from-a-pdf-through-python-and-tabula-py-6f642a9ee673
box = [12.8,1,18.0,27]
fc = 28.28
for i in range(0, len(box)):
    box[i] *= fc

df  = tabula.read_pdf(Form1099_path, encoding='latin-1', pages = '3', area=[box])[0]  
df

In [ ]:
# SHORT TERM PAGES
df_transactios = pd.DataFrame(columns = ['Security', 'Option', 'Date_Sold', 'Quantity', 'GrossSale',
                                         'DateAdquired', 'Cost', 'Washsale','Lost', 'Comment'])
                                        #'Washsale_Code','Washsale_Value', 
df_transactios

box0  = [12.3,1,18.0,27]
box1  = [5.20,1,18.0,27] 
box2  = [5.20,1,11.2,27] 
fc = 28.28

for i in range(0, len(box)):
    box0[i] *= fc 
    box1[i] *= fc
    box2[i] *= fc

NPST = 5 #5
NPLT = 52
NP   = NPLT - NPST  # Is the Number of page where LONGterm starst - N of pag od Shortterm stars
for Pages in range(NP):
    j = NPST+Pages
    if (j == NPST):  box = box0
    if (j > NPST):   box = box1
    #if (j >= NPST):   box = box1
    if (j == NPLT): box = box2
    df1  = tabula.read_pdf(Form1099_path, encoding='latin-1', pages = str(j), area=[box])[0]  
    #df1
    df_transactios = ReadPage( df1, df_transactios)
    

In [ ]:
#df1
lendf_ST = len(df_transactios)
df_transactios = WashSaleColumns(df_transactios)
df_transactios

In [ ]:
df_LT_transactios = pd.DataFrame(columns = ['Security','Option','Date_Sold', 'Quantity', 'GrossSale',
                                         'DateAdquired', 'Cost', 'Washsale', 'Lost', 'Comment'])
df_LT_transactios

# LONG TERM PAGES
box0  = [5.20,1,18.0,27] #[14.4,1,18.0,27]
box1  = [5.20,1,18.0,27] 
box2  = [5.20,1,11.2,27] 
fc    = 28.28

for i in range(0, len(box)):
    box0[i] *= fc 
    box1[i] *= fc
    box2[i] *= fc

NPST = 52 #5
NPLT = 54
NP   = NPLT - NPST  # Is the Number of page where LONGterm starst - N of pag od Shortterm stars
for Pages in range(NP):
    j = NPST+Pages
    if (j == NPST):  box = box0
    if (j > NPST):   box = box1
    #if (j >= NPST):   box = box1
    if (j == NPLT): box = box2
    df1  = tabula.read_pdf(Form1099_path, encoding='latin-1', pages = str(j), area=[box])[0]  
    #df1
    df_LT_transactios = ReadPage( df1, df_LT_transactios)

In [ ]:
lendf_LT =len(df_LT_transactios)

df_LT_transactios = WashSaleColumns(df_LT_transactios)
df_LT_transactios

df_LT_transactios #.head()

In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen.canvas import Canvas

# Change the path below to the correct path for your computer.
pdf_path = (
    Path.home() 
    / "TaxesForms"
    / "2022TaxYear"
    / "f8949.pdf"
)
print (pdf_path)



In [ ]:
df_transactios.columns, df_LT_transactios.columns

In [ ]:
xcoor  = 10   #To be changed according to your pdf
ycoor  = 390  #To be changed according to your pdf
DeltaX = 20
DeltaY = 24
#PAGE 2
xcoor2 = 10   #To be changed according to your pdf
ycoor2 = 425  #To be changed according to your pdf


# PageNumber : page of form 0 or 1
# FormNumber : 0, 1, 2

Ncycles1 = np.floor_divide( lendf_ST ,14) + 1
Ncycles2 = np.floor_divide( lendf_LT ,14)
print (Ncycles1, Ncycles2)

FormNumber = 0
PageNumber = 0

for i in range(Ncycles1): 
    ## FOR EACH PAGE
    # Path to where you want the files to be storage:
    #
    Out_pdf_path    = '/Users/YourUserName/TaxesForms/2022TaxYear/f8949_F'+str(i+1)+'_Page1.pdf' 
    Out_pdf_path_F1 =  '/Users/YourUserName/TaxesForms/2022TaxYear/f8949_F'+str(i+1)+'_Page1n2.pdf' 

    FillPage(pdf_path, Out_pdf_path, xcoor, ycoor, DeltaX, DeltaY, df_transactios, PageNumber, FormNumber, lendf_ST)
    if (FormNumber<2):
        FillPage(Out_pdf_path, Out_pdf_path_F1, xcoor2, ycoor2, DeltaX, DeltaY, df_LT_transactios, PageNumber+1, FormNumber, lendf_LT)

    FormNumber += 1

# Schwabs Acc

- If you don't have any other transactions file (F1099) you can skip this part.
- We can also add any other transactions, for instance in this case, we are adding transactions from an csv file provided by Schwabs. 
- If you have a different provider you may need to check the structure of the file, see which columns are provided and which ones you may need to calculate yourself. 

In [ ]:
FormShwabs_path = (
    Path.home()
    / "your-path-to-files"
    / "TaxesForms"
    / "2022TaxYear"
    / "ShwabsF1099_2022.csv"
)

dfShwabsST = pd.read_csv(FormShwabs_path, sep=",", header = 50, engine='python') 
ldfShwabsST   = len(dfShwabsST)
dfShwabsST.columns = ['Security', 'DateAdquired', 'Date_Sold',
       'GrossSale',  'Cost', 'AccruedDiscount', 'WashSaleDisallowed', 'DURATION',
       'F8949Code', 'QOF', 'FedTaxWithheld', 'Covered', 'ReportedIRS', 'LossAllowed1d',
       'Profit', 'UnrealizedProfit21', 'UnrealizedProfit22', 'AggregateProfitContracts',
       'Reported','Bartering', 'State', 'StateID','StateTaxWithheld','Option']
print (dfShwabsST.columns)


dfOption = pd.DataFrame( ['']*ldfShwabsST, columns = {'Option'})
dfQntt = pd.DataFrame( ['']*ldfShwabsST, columns = {'Quantity'})
dfSymbol = pd.DataFrame( ['']*ldfShwabsST, columns = {'Symbol'})
dfDA = pd.DataFrame( ['']*ldfShwabsST, columns = {'DateAdquired'})
dfDS = pd.DataFrame( ['']*ldfShwabsST, columns = {'Date_Sold'})

for i in range(ldfShwabsST ):
    text = str(dfShwabsST['Security'][i])[5:9]
    if (text=='CALL' or text=='PUT '):
        dfOption['Option'][i] = text
        dfSymbol['Symbol'][i] = str(dfShwabsST['Security'][i])[9:16] 
    else:
        dfOption['Option'][i] = '' 
        dfSymbol['Symbol'][i] = str(dfShwabsST['Security'][i])[5:12]
    
    text = str(dfShwabsST['Security'][i])[0:4]
    dfQntt['Quantity'][i] = float(text )
    dfDA['DateAdquired'][i] = str(dfShwabsST['DateAdquired'][i]).replace('/2022','/22')  
    dfDS['Date_Sold'][i]    = str(dfShwabsST['Date_Sold'][i]).replace('/2022','/22')  
    
    
dfShwabsST['Option']   = dfOption
dfShwabsST['Quantity'] = dfQntt
dfShwabsST['Security'] = dfSymbol
dfShwabsST['DateAdquired']= dfDA
dfShwabsST['Date_Sold']=dfDS
dfShwabsST

In [ ]:
dfShwabsST['Lost'] = dfShwabsST['GrossSale'] - dfShwabsST['Cost']
dfShwabsST['Washsale_Code'] = ['']*ldfShwabsST
dfShwabsST['Washsale_Value'] = np.zeros(ldfShwabsST) 
dfShwabsST

In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen.canvas import Canvas

# Change the path below to the correct path for your computer.
pdf_path = (
    Path.home()
    / "your-path-to-files"
    / "TaxesForms"
    / "2022TaxYear"
    / "f8949.pdf"
)
print (pdf_path)

xcoor  = 10   #To be changed according to your pdf
ycoor  = 390  #To be changed according to your pdf
DeltaX = 20
DeltaY = 24

# PageNumber : page of form 0 or 1
# FormNumber : 0, 1, 2

Ncycles1 = np.floor_divide( ldfShwabsST ,14) + 1
#Ncycles2 = np.floor_divide( ldfCripto_LT ,14)
print (Ncycles1) # , Ncycles2

In [ ]:
FormNumber = 0
PageNumber = 0

for i in range(Ncycles1):    
    ## FOR EACH PAGE
    Out_pdf_path    = '/Users/yourpathtofiles/TaxesForms/2022TaxYear/f8949_Sch'+str(i+1)+'_Page1.pdf' 
    # If you have transactions to fill in the long term you may need to uncomment the following lines:
    #Out_pdf_path_F1 =  '/Users/yourpathtofiles/TaxesForms/f8949_B_F'+str(i+1)+'_Page1n2.pdf' 
 
    FillPage(pdf_path, Out_pdf_path, xcoor, ycoor, DeltaX, DeltaY, dfShwabsST, PageNumber, FormNumber, ldfShwabsST)
    # If you have transactions to fill in the long term you may need to uncomment the following lines:
    # note that here 4, is the number of pages that will fill. You may need to change that value.
    #if (FormNumber<4):
    #    FillPage(Out_pdf_path, Out_pdf_path_F1, xcoor2, ycoor2, DeltaX, DeltaY, df_LT_transactios, PageNumber+1, FormNumber, lendf_LT)

    FormNumber += 1

In [ ]:
GrosSale_LT  = np.sum(df_LT_transactios['GrossSale'])
CostBasis_LT = np.sum(df_LT_transactios['Cost'])
GainLost_LT     = np.sum(df_LT_transactios['Lost'])


print ('\n__________________________________________________________________________________ ')
print ('\nLONG TERM')
print ('Gross Sale.  Cost Basis.     Net Gain or Loss(-)')
print (  f' {GrosSale_LT:8,.2f}',
         f'   {CostBasis_LT:10,.2f}',
         f'   {GainLost_LT:10,.2f}'
    )

GrosSale_ST  = np.sum(df_transactios['GrossSale'])
CostBasis_ST = np.sum(df_transactios['Cost'])
GainLost_ST  = np.sum(df_transactios['Lost'])
Washsale_ST  = np.sum(df_transactios['Washsale_Value'])

print ('\n__________________________________________________________________________________ ')
print ('\n\nSHORT TERM')
print ('Gross Sale.  Cost Basis.     Net Gain or Loss(-)   Wash Sale')
print (  f' {GrosSale_ST:8,.2f}',
         f'   {CostBasis_ST:10,.2f}',
         f'   {GainLost_ST:12,.2f}',
         f'   {Washsale_ST:14,.2f}'
    )

#print (np.sum(df_LT_transactios['Cost']))


In [ ]:
GrosSale_CS  = np.sum(dfShwabsST['GrossSale'])
CostBasis_CS = np.sum(dfShwabsST['Cost'])
GainLost_CS     = np.sum(dfShwabsST['Lost'])


print ('\n__________________________________________________________________________________ ')
print ('\nSHORT TERM SHWABS')
print ('Gross Sale.  Cost Basis.     Net Gain or Loss(-)')
print (  f' {GrosSale_CS:8,.2f}',
         f'   {CostBasis_CS:10,.2f}',
         f'   {GainLost_CS:10,.2f}'
    )


In [ ]:
# Short Term
Box_A_h = GrosSale_ST - CostBasis_ST +  Washsale_ST
Box_A_h

In [ ]:
# Only Robinhood
GainLost_ST + GainLost_LT

In [ ]:
#Schedule D - Equities Robinhood And SCHWABS - LINE 7
Colum_Ah_Sum = Box_A_h + GainLost_CS
Colum_Ah_Sum

In [ ]:
# Schedule D - LINE 15 - Long Term
Colum_Bh_Sum = GrosSale_LT - CostBasis_LT 
Colum_Bh_Sum

In [ ]:
# Schedule D - LINE 16

Colum_Ah_Sum + Colum_Bh_Sum